# STATIC ANALYSIS 

In [1]:
from typing import Dict
import yaml
import json
import glob
import os
import requests
from requests.structures import CaseInsensitiveDict
import string
import random
from collections import defaultdict
import ast
import time
import pandas as pd
import urllib.parse
import threading
from time import sleep
from threading import Thread
import shutil

In [2]:
# Function CURL GET
def curl_get(url: str,MOBSF_API_KEY):
    headers = CaseInsensitiveDict()
    headers["X-Mobsf-Api-Key"] = MOBSF_API_KEY
    response = requests.get(url, headers = headers)
    return byteStringToDict(response.content),response.status_code

In [3]:
# Function CURL POST
def curl_post(url, data, MOBSF_API_KEY):
    headers = CaseInsensitiveDict()
    #headers["Content-Type"] = "application/json"
    headers["X-Mobsf-Api-Key"] = MOBSF_API_KEY
    response = requests.post(url, data, headers=headers)
    return stringToDict(response.text),response.status_code

In [4]:
# Function UPLOAD file curl POST
def curl_upload_post(url, file_name, file_path, MOBSF_API_KEY):
    payload = {}
    files=[
          ('file',(file_name,open(file_path,'rb'),'application/octet-stream'))
    ]
    headers = {
      'X-Mobsf-Api-Key': MOBSF_API_KEY
    }
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return stringToDict(response.text),response.status_code

In [5]:
# Function Byte String to Dict
def byteStringToDict(byteString: bytes):
    dict_str = byteString.decode("UTF-8")
    mydata = ast.literal_eval(dict_str)
    return mydata

In [6]:
# Function String to Dict
def stringToDict(stringText: str):
    res = json.loads(stringText)
    return res

In [7]:
# Function Dict to JSON
def dictToJson(dictionary: dict):
    json_object = json.dumps(dictionary, indent = 4) 
    return json_object

In [8]:
# Function Write JSON to FIle
def writeJSONToFile(filename: str, jsonString: str):
    with open(filename, 'w') as outfile:
        outfile.write(jsonString)

In [9]:
# Function convert list to string
def listToString(listData: list):
    listToStr = '\n'.join([str(elem) for elem in listData])
    return listToStr

In [10]:
# Class Generate APK DATA FORM for CURL POST
class CurlPostDataForm(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value)

In [11]:
def checkFileOrDirectoryExist(path):
    if os.path.exists(path):
        return True
    else:
        return False

In [12]:
def prepareEnvironment(apk_file_path: str,directory_name: str):
    check_directory_exist = checkFileOrDirectoryExist(apk_file_path+"static_analysis")
    if(check_directory_exist == False):
        os.mkdir(apk_file_path+"static_analysis")
        return "Mkdir directory static_analysis success!"
    else:
        return "Directory is static_analysis existed!"

In [13]:
# CONSTANT VALUE
MOBSF_SERVER = "192.168.1.20"
#MOBSF_SERVER = "193.206.183.20"
MOBSF_PORT = "8000"
MOBSF_API_KEY = "fa81deec8c83e903fa85dab8e3e3a7b1f4ac191f2bf1ba984be01b14b9698e66"
#MOBSF_API_KEY = "345452a72a7b1a8935e712ed06364bda91d1c622a774210115a7c0c43c66929e"
category_name = "photography"
apk_scan_url = "http://"+MOBSF_SERVER+":"+MOBSF_PORT+"/api/v1/scans?page=1&page_size=10"
apk_generate_JSON_report_url = "http://"+MOBSF_SERVER+":"+MOBSF_PORT+"/api/v1/report_json"
apk_scan_file_url = "http://"+MOBSF_SERVER+":"+MOBSF_PORT+"/api/v1/scan"
apk_upload_url = "http://"+MOBSF_SERVER+":"+MOBSF_PORT+"/api/v1/upload"
apk_delete_scan_url = "http://"+MOBSF_SERVER+":"+MOBSF_PORT+"/api/v1/delete_scan"
#apk_file_path = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\"
apk_file_path = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\photography_category\\"
#static_analysis_report = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\static_analysis\\"
static_analysis_report = "C:\\Users\\ASUS\\anaconda3\metadata-gdpr-client\\static_analysis\\"
result_csv_path = "C:\\Users\\ASUS\\anaconda3\\metadata-gdpr-client\\leaking_list_report.csv"

In [14]:
# 1. Read CSV file
# 1.0 Prepare environment
create_directory = prepareEnvironment(apk_file_path,"static_analysis")
print(create_directory)
# 1.1 Read CSV file
df = ""
#check_file_exist = checkFileOrDirectoryExist("C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\"+category_name+"_static_analysis.csv")
check_file_exist = checkFileOrDirectoryExist("C:\\Users\\ASUS\\anaconda3\\metadata-gdpr-client\\leaking_list_report_static_analysis.csv")
if(check_file_exist):
    #df = pd.read_csv("C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\"+category_name+"_static_analysis.csv",low_memory=False)
    df = pd.read_csv("C:\\Users\\ASUS\\anaconda3\\metadata-gdpr-client\\leaking_list_report_static_analysis.csv",low_memory=False)
    print("Reading file "+category_name+"_static_analysis.csv")
else:
    #df = pd.read_csv("C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\"+category_name+"_apk_popular.csv",low_memory=False)
    df = pd.read_csv(result_csv_path,low_memory=False)
    print("Reading file "+category_name+"_apk_popular.csv")
#drop_duplicates = df.drop_duplicates(subset=['pkg_name'])
#pkg_name_arr = drop_duplicates["pkg_name"]
pkg_name_arr = df["pkg_name"]
#apkfile_name_arr = drop_duplicates["apkfile_name"]
apkfile_name_arr = df["apkfile_name"]
#pkg_name_arr = drop_duplicates["pkg_name"]
#apkfile_name_arr = drop_duplicates["apkfile_name"]
# 1.2 Check completed static analysis
#static_analysis_done_index = drop_duplicates[(drop_duplicates.manifest_permission.isnull())].index[0]
static_analysis_done_index = df[(df.manifest_permission.isnull())].index[0]
print("Static analysis done index: "+str(static_analysis_done_index))

Directory is static_analysis existed!
Reading file photography_static_analysis.csv
Static analysis done index: 985


In [15]:
# 2. Static Analysis
len_apkfile_name_arr=len(apkfile_name_arr)
#for apk_num in range(static_analysis_done_index,len_apkfile_name_arr):
for apk_num in range(1092,len_apkfile_name_arr):
    try:
        print("----------------------------------------------["+str(apk_num)+"]----------------------------------------------")
        apkfile_name = apkfile_name_arr[apk_num]
        print("Apk file name: "+apkfile_name)
        index = int(df[df['apkfile_name']==apkfile_name].index.values)
        #print(index)
        # 2.1 Upload file to MOBSF server
        check_apk_exist = checkFileOrDirectoryExist(apk_file_path+apkfile_name)
        upload_response = ""
        upload_code = 0
        if(check_apk_exist):
            print("File "+apkfile_name+" founded!")
            upload_response,upload_code = curl_upload_post(apk_upload_url,apkfile_name,apk_file_path+apkfile_name, MOBSF_API_KEY)
        else:
            print("APK file "+apkfile_name+" not found!")
            #drop_duplicates.loc[index,"manifest_permission"]="APK not found!"
            df.loc[index,"manifest_permission"]="APK not found!"
            continue
        apk_upload_hash = ""
        scan_type = ""
        apk_permission = []
        if(upload_code==200):
            apk_upload_hash = upload_response['hash']
            scan_type = upload_response['scan_type']
            print("Upload file "+apkfile_name+" success!")
        else:
            print("Upload file "+apkfile_name+" error!")
            #drop_duplicates.loc[index,"manifest_permission"]="Upload file failed!"
            df.loc[index,"manifest_permission"]="Upload file failed!"
            continue
        #print(apk_upload_hash)
        #print(scan_type)
        # 2.2 Scan uploaded file
        scan_data = CurlPostDataForm()
        scan_data["scan_type"] = scan_type
        scan_data["file_name"] = apkfile_name
        scan_data["hash"] = apk_upload_hash
        #print(scan_data)
        scan_response,scan_code = curl_post(apk_scan_file_url,scan_data, MOBSF_API_KEY)
    #     print(scan_code)
    #     print(type(scan_code))
    #     apk_permission = list(scan_response["permissions"].keys())
        #time.sleep(400)
        if(scan_code==200):
            apk_permission = list(scan_response["permissions"].keys())
            print("Static analysis "+apkfile_name+" completed!")
        else:
            print("Static analysis "+apkfile_name+" failed!")
            #drop_duplicates.loc[index,"manifest_permission"]="Static analysis failed!"
            df.loc[index,"manifest_permission"]="Static analysis failed!"
            delete_data = CurlPostDataForm()
            delete_data["hash"] = apk_upload_hash
            delete_response,delete_code = curl_post(apk_delete_scan_url, delete_data, MOBSF_API_KEY)
            continue
        print(apk_permission)
        # 2.3 Write static analysis report to JSON
        writeJSONToFile(static_analysis_report+apkfile_name+".json",dictToJson(scan_response))
        # 2.4 Update csv permission
        if(len(apk_permission)==0):
            #drop_duplicates.loc[index,"manifest_permission"]="Not found permission"
            df.loc[index,"manifest_permission"]="Not found permission"
        else:
            #drop_duplicates.loc[index,"manifest_permission"]=listToString(apk_permission)
            df.loc[index,"manifest_permission"]=listToString(apk_permission)
        #drop_duplicates.loc[index,"manifest_permission"]=listToString(apk_permission)
        # 2.5 Write file
        #drop_duplicates.to_csv("C:\\Users\\ASUS\\anaconda3\\phd_implement\\"+category_name+"_category\\"+category_name+"_static_analysis.csv",index=False)
        df.to_csv("C:\\Users\\ASUS\\anaconda3\\metadata-gdpr-client\\leaking_list_report_static_analysis.csv",index=False)
        # 2.5 Delete completed static analysis
        delete_data = CurlPostDataForm()
        delete_data["hash"] = apk_upload_hash
        delete_response,delete_code = curl_post(apk_delete_scan_url, delete_data, MOBSF_API_KEY)
        if(delete_code==200):
            print("Deleted scan file "+apkfile_name+" success!")
        else:
            print("Deleted scan file "+apkfile_name+" error!")
            break
    except Exception as e:
        df.loc[df['apkfile_name'] == apkfile_name, 'manifest_permission'] = "Duplicated"
        continue
    #break

----------------------------------------------[1092]----------------------------------------------
Apk file name: LINE_13.13.2_apkcombo.com.apk
File LINE_13.13.2_apkcombo.com.apk founded!
Upload file LINE_13.13.2_apkcombo.com.apk success!
Static analysis LINE_13.13.2_apkcombo.com.apk completed!
['android.permission.CAMERA', 'android.permission.VIBRATE', 'android.permission.READ_CONTACTS', 'android.permission.INTERNET', 'android.permission.READ_PHONE_STATE', 'android.permission.READ_PHONE_NUMBERS', 'android.permission.CALL_PHONE', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.ACCESS_FINE_LOCATION', 'android.permission.READ_MEDIA_VIDEO', 'android.permission.READ_MEDIA_IMAGES', 'android.permission.READ_MEDIA_AUDIO', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.ACCESS_WIFI_STATE', 'android.permission.CHANGE_WIFI_STATE', 'android.permission.POST_NOTIFICATIONS', 'com.